高斯平滑我理解为，利用高斯滤波器，使卷积核周围的像素点将影响锚点位置的像素，且核越大，影响越大，图像越模糊。sigma取值越大，图像越平缓（模糊）。

cv2.GaussianBlur用于python中的高斯平滑。

dst = cv2.GaussianBlur(src,ksize,sigmax);
src表示原始图像，ksize表示核大小，sigmaX表示X方向方差。核大小（N, N）必须是奇数，X方向方差主要控制权重。

In [1]:
import cv2
#import numpy as np

image = cv2.imread("img3.jpg",0)
#高斯平滑
blurImage = cv2.GaussianBlur(image,(51,51),5)
cv2.imshow("src",image)
cv2.imshow("GaussBlur",blurImage)
cv2.waitKey(0)
cv2.destroyAllWindows()

下面是构建高斯卷积函数原理，bug了。。  下面都是算法原理，上面是opencv内置函数可以直接调用

In [2]:
#构建高斯卷积算子
import numpy as np
import math

def getGaussKernel(sigma,H,W):
    #第一步：构建高斯矩阵
    gaussMatrix = np.zeros([H,W],np.float32)
    #得到中心点位置
    cH = (H-1)/2
    cW = (W-1)/2
    #计算gauss(sigma,r,c)
    for r in xrange(H):
        for c in xrange(W):
            norm2 = math.pow(r-cH,2) + math.pow(c-cW,2)
            #math.pow(a,b)=a^b
            gaussMatrix[r],[c] = math.exp(-norm2/(2*math.pow(sigma,2))) 
            #math.exp(a)=e^a
    #第二步，计算高斯和
    sumGM = np.sum(gaussMatrix)
    #第三步，归一化
    gaussKernel = gaussMatrix/sumGM
    return gaussKernel

#实现高斯平滑
# -*- coding: utf-8 -*-
import sys
import numpy as np
from scipy import signal
import cv2
#高斯平滑，返回的数据类型为浮点型
def gaussBlur(image,sigma,H,W,_boundary = 'fill',_fillvalue = 0):
    '''
    #构建HxW的高斯平滑算子
    gaussKernelxy = getGaussKernel(sigma,H,W)
    #图像矩阵和高斯卷积核卷积
    gaussBlur_xy = signal.convolve2d(image,gaussKernelxy,mode='same',boundary = _boundary,fillvalue=_fillvalue)
    return gaussBlur_xy
    '''
    #因为高斯核是可分解的，根据卷积的结合律
    #先进行水平方向的卷积，然后再进行垂直方向的卷积
    gaussKenrnel_x = getGaussKernel(sigma,1,W)
    gaussBlur_x = signal.convolve2d(image,gaussKenrnel_x,mode='same',boundary = _boundary,fillvalue=_fillvalue)
    gaussKenrnel_y = getGaussKernel(sigma,H,1)
    gaussBlur_xy = signal.convolve2d(gaussBlur_x,gaussKenrnel_y,mode='same',boundary = _boundary,fillvalue=_fillvalue)
    return gaussBlur_xy

image2 = cv2.imread("img3.jpg",0)
cv2.imshow("image2",image2)
#cv2.imwrite("img4.jpg",image)
# 3 11 11 9 25 25
blurImage2 = gaussBlur(image2,5,51,51,'symm')
'''
#如果输入的图像是8位图,输出的
blurImage = np.round(blurImage)
blurImage = blurImage.astype(np.uint8)
cv2.imshow("gaussBlur",blurImage)
cv2.imwrite("gaussBlur.png",blurImage)
'''
#如果输入的图像数据类型是浮点型，且像素值归一到[0,1]
image_0_1 = image2/255.0
blurImage_0_1 = gaussBlur(image_0_1,4,5,5,'symm')
cv2.imshow("gaussBlur-0-1",blurImage_0_1)
cv2.waitKey(0)
cv2.destroyAllWindows()

TypeError: 'float' object is not iterable